In [56]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/joaquin/Documents/GitHub/skforecast


In [57]:
from skforecast.recursive import ForecasterRecursive
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [58]:
series = pd.Series(np.arange(10000, dtype=float), index=pd.date_range("2020-01-01", periods=10000, freq="h"))
series_original = series.copy()
y = series.to_numpy()
forecaster = ForecasterRecursive(
    regressor=LinearRegression(),
    transformer_y=StandardScaler(),
    lags=200
)

In [20]:
# x, y = forecaster._create_lags(
#     y=series
# )
# x.astype(np.float32)

In [21]:
# x_new, y_new = forecaster._create_lags_new(
#     y=series
# )
# x_new

In [34]:
%%timeit
forecaster.fit(y=series)

217 ms ± 13.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
%%timeit
forecaster.fit(y=series)

215 ms ± 7.69 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
%%timeit
_create_lags(
    forecaster,
    y
)

3.73 ms ± 422 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
%%timeit
forecaster._create_lags_deprecated(
    y
)

3.84 ms ± 451 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [25]:
%%timeit
forecaster._create_lags(
    y
)

1.89 ms ± 195 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [46]:
from skforecast.recursive import ForecasterRecursiveMultiSeries

forecaster = ForecasterRecursiveMultiSeries(
    regressor=LinearRegression(),
    lags=3,
)

series = {f"series_{i}": pd.Series(np.arange(1000, dtype=float), index=pd.date_range("2020-01-01", periods=1000)) for i in range(1000)}

╭───────────────────────────── DataTransformationWarning ──────────────────────────────╮
│ When using a linear model, it is recommended to use a transformer_series to ensure   │
│ all series are in the same scale. You can use, for example, a `StandardScaler` from  │
│ sklearn.preprocessing.                                                               │
│                                                                                      │
│ Category : skforecast.exceptions.DataTransformationWarning                           │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/recursive/_forecaster_recursive │
│ _multiseries.py:486                                                                  │
│ Suppress : warnings.simplefilter('ignore', category=DataTransformationWarning)       │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [42]:
%%timeit

for y in series.values():
    forecaster._create_lags_deprecated(
        y
    )

612 ms ± 157 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [43]:
%%timeit

for y in series.values():
    forecaster._create_lags(
        y
    )

244 ms ± 62.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [45]:
%%timeit

forecaster.fit(series=series)

6.59 s ± 95.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [47]:
%%timeit

forecaster.fit(series=series)

5.92 s ± 117 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [73]:
from skforecast.direct import ForecasterDirect

forecaster = ForecasterDirect(
    regressor=LinearRegression(),
    steps=24,
    lags=50
)
y = pd.Series(np.arange(1000, dtype=float), index=pd.date_range("2020-01-01", periods=1000))


In [74]:
%%timeit
forecaster._create_lags_deprecated(
    y
)

10.2 ms ± 1.3 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [75]:
%%timeit
forecaster._create_lags(
    y
)

409 μs ± 56.1 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [76]:
%%timeit
forecaster._create_lags_fast(
    y
)

176 μs ± 8.57 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [71]:
x_, y_ = forecaster._create_lags_fast(
    y
)
x_

array([[  2.,   1.,   0.],
       [  3.,   2.,   1.],
       [  4.,   3.,   2.],
       ...,
       [992., 991., 990.],
       [993., 992., 991.],
       [994., 993., 992.]], shape=(993, 3))

In [72]:
forecaster._create_lags_deprecated(
    y
)

(array([[  2.,   1.,   0.],
        [  3.,   2.,   1.],
        [  4.,   3.,   2.],
        ...,
        [992., 991., 990.],
        [993., 992., 991.],
        [994., 993., 992.]], shape=(993, 3)),
 array([[  3.,   4.,   5.,   6.,   7.],
        [  4.,   5.,   6.,   7.,   8.],
        [  5.,   6.,   7.,   8.,   9.],
        ...,
        [993., 994., 995., 996., 997.],
        [994., 995., 996., 997., 998.],
        [995., 996., 997., 998., 999.]], shape=(993, 5)))